In [57]:
import pandas as pd
import random

from faker import Faker

import sqlite3

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [58]:
N_PRODUTOS = 100
N_CLIENTES = 20
N_COMPRAS = 500

In [59]:
faker = Faker('pt_BR')
faker.name()

'Sofia Teixeira'

In [60]:
db = sqlite3.connect('TesteSQL.db')
db

In [61]:
%sql sqlite:///TesteSQL.db

In [62]:
produtos = pd.DataFrame([[f'prod {i:02}', round(random.random() * 10, 2)] 
                         for i in range(N_PRODUTOS)], 
                        columns=['produto', 'preço'])
produtos.head()

,produto,preço
0,prod 00,1.00
1,prod 01,7.85
2,prod 02,7.56
3,prod 03,6.07
4,prod 04,6.55


In [63]:
produtos.to_sql('produtos', db, index_label='id', if_exists='replace')

In [64]:
pd.read_sql("SELECT * FROM produtos;", db, 'id').head()

,produto,preço
id,,
0,prod 00,1.00
1,prod 01,7.85
2,prod 02,7.56
3,prod 03,6.07
4,prod 04,6.55


In [65]:
clientes = pd.DataFrame([
    [faker.name(), faker.free_email(), random.randint(18, 60)]
for _ in range(N_CLIENTES)], columns=['nome', 'email', 'idade'])

clientes.head()

,nome,email,idade
0,Dra. Bruna das Neves,ealmeida@gmail.com,54
1,Vitor Gabriel Cavalcanti,catarinamoreira@hotmail.com,22
2,Luiz Otávio Campos,bcunha@uol.com.br,25
3,Alexandre Santos,fpereira@yahoo.com.br,29
4,Dr. Thomas Almeida,fariaslucas-gabriel@hotmail.com,37


In [66]:
clientes.to_sql('clientes', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM clientes;", db, 'id').head()

,nome,email,idade
id,,,
0,Dra. Bruna das Neves,ealmeida@gmail.com,54
1,Vitor Gabriel Cavalcanti,catarinamoreira@hotmail.com,22
2,Luiz Otávio Campos,bcunha@uol.com.br,25
3,Alexandre Santos,fpereira@yahoo.com.br,29
4,Dr. Thomas Almeida,fariaslucas-gabriel@hotmail.com,37


In [67]:
compras = pd.DataFrame([[
    random.randint(0, N_CLIENTES - 1),
    random.randint(0, N_PRODUTOS - 1)
] for _ in range(N_COMPRAS)], columns=['id_cliente', 'id_produto'])
compras.head()

,id_cliente,id_produto
0,6,62
1,14,16
2,3,11
3,14,12
4,9,8


In [68]:
compras.to_sql('compras', db, index_label='id', if_exists='replace')
pd.read_sql("SELECT * FROM compras;", db, 'id').head()

,id_cliente,id_produto
id,,
0,6,62
1,14,16
2,3,11
3,14,12
4,9,8


In [69]:
del compras, clientes, produtos

# Select

Ao fazer uma extração num banco de dados SQL, independentemente do 
DBMS (se é Oracle, SQLite, MySQL/MariaDB e etc.), você sempre vai utilizar a 
palava `SELECT` seguida pelo que você quer trazer, e por fim a palavra FROM
informando de onde você irá coletar esses dados, por exemplo:

In [70]:
%%sql

SELECT nome, idade FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,idade
Dra. Bruna das Neves,54
Vitor Gabriel Cavalcanti,22
Luiz Otávio Campos,25
Alexandre Santos,29
Dr. Thomas Almeida,37
Manuela Lima,57
Ana Júlia Vieira,18
Igor Dias,28
Breno Ribeiro,40
Beatriz Pinto,54


Nesse caso, eu estou trazendo o nome e idade de todos os clientes da base.

# Limit

Geralmente não queremos trazer todos os dados de uma vez, muitas vezes os dados
que guardamos nas tabelas do banco de dados iriam facilmente estourar a  memória
dos nossos servidores, para limitar a pesquisa, existem vários métodos, o mais 
simples que tem é o `LIMIT n`, onde você informa quantos registros você quer ver
no máximo.

In [71]:
%%sql

SELECT * FROM clientes LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
0,Dra. Bruna das Neves,ealmeida@gmail.com,54
1,Vitor Gabriel Cavalcanti,catarinamoreira@hotmail.com,22
2,Luiz Otávio Campos,bcunha@uol.com.br,25
3,Alexandre Santos,fpereira@yahoo.com.br,29
4,Dr. Thomas Almeida,fariaslucas-gabriel@hotmail.com,37


Nesse caso, eu trouxe 5 registros da tabela cliente.

<span style="color:red"><b>Em uma aplicação, sempre use o</span></b> `LIMIT` 
<span style="color:red"><b>com um</span></b> `ORDER BY` 
<span style="color:red"><b>quer será explicado mais a frente

## Exercício 01

Traga 10 produtos e seu respectivo preço

In [72]:
%%sql

SELECT * FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
0,prod 00,1.0
1,prod 01,7.85
2,prod 02,7.56
3,prod 03,6.07
4,prod 04,6.55
5,prod 05,6.87
6,prod 06,5.37
7,prod 07,0.75
8,prod 08,4.23
9,prod 09,7.78


# Projeção

Também posso operar sobre os valores retornados, por exemplo, se ao invés da idade
da pessoa, eu quiser saber somente a faixa etária dela, posso fazer algum cálculo
no select e assim trazer esse dado já trabalhado do próprio banco de dados. 

In [73]:
%%sql

SELECT NOME, ROUND(idade / 10) * 10 AS FAIXA_ETARIA
FROM clientes

 * sqlite:///TesteSQL.db
Done.


nome,FAIXA_ETARIA
Dra. Bruna das Neves,50.0
Vitor Gabriel Cavalcanti,20.0
Luiz Otávio Campos,20.0
Alexandre Santos,20.0
Dr. Thomas Almeida,30.0
Manuela Lima,50.0
Ana Júlia Vieira,10.0
Igor Dias,20.0
Breno Ribeiro,40.0
Beatriz Pinto,50.0


Essa ação de selecionar um dado trabalhado, recebe vários nomes, como mapeamento
ou projeção.

## Exercício 2

Traga 10 produtos e seu preço convertido para dólar, considere a cotação do dólar
com sendo US\\$1,00=R\\$5,20

In [74]:
%%sql

SELECT produto, (preço/5.20)  AS preço_convertido
FROM produtos LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço_convertido
prod 00,0.1923076923076923
prod 01,1.5096153846153846
prod 02,1.4538461538461538
prod 03,1.1673076923076924
prod 04,1.2596153846153846
prod 05,1.3211538461538461
prod 06,1.0326923076923076
prod 07,0.14423076923076922
prod 08,0.8134615384615386
prod 09,1.4961538461538462


# Where

O `LIMIT` como dito anteriormente é a forma mais simples de reduzir a quantidade
de dados trazidos em uma pesquisa, mas a forma mais comum de se usar é o `WHERE`.

Geralmente quando fazemos uma consulta no banco de dados, não precisamos de todos
os dados da tabela, somente dados que respeitem algum requisito, como por exemplo,
no nosso caso, se a gente quiser saber quais clientes temos com menos de 30 anos:

In [75]:
%%sql

SELECT nome, idade FROM clientes WHERE idade < 30

 * sqlite:///TesteSQL.db
Done.


nome,idade
Vitor Gabriel Cavalcanti,22
Luiz Otávio Campos,25
Alexandre Santos,29
Ana Júlia Vieira,18
Igor Dias,28
Rafaela Rezende,28
Sr. Nathan Novaes,21


## Exercício 3

Traga todos os produtos que custem menos do que 1 real

In [76]:
%%sql

SELECT produto, preço FROM produtos WHERE preço<1

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 07,0.75
prod 20,0.32
prod 23,0.91
prod 29,0.61
prod 31,0.56
prod 61,0.43
prod 63,0.01
prod 73,0.54
prod 85,0.07
prod 87,0.24


# Join

Uma das principais características dos bancos relacionais é justamente a sua capacidade
de ter relacionamento entre os dados.

Geralmente esse relacionamento é feito utilizando chaves primárias (PK). Geralmente
as PKs são representadas por um número inteiro chamado ID, mas não necessariamente,
por exemplo, um cadastro de pessoas pode ter como PK o CPF da pessoa, o que importa
é que não haja repetição de valores.

As PKs também podem ter mais de uma coluna, por exemplo, um supermercado pode ter
como PK um produto e a marca dele.

Mas por hora iremos abstrair essas PKs menos ortodoxas e focar nas padrões que é só
uma coluna ID.

As PKs geralmente são utilizadas para formar relacionamento entre as tabelas, ou seja,
a partir do momento que eu cadastrei um cliente no meu sistema, e gerei um ID para ele
toda as interações dele serão registradas utilizando o seu ID e não mais dados
pessoais.

No nosso caso, registramos as compras utilizando o ID do cliente e do produto, como
podemos ver abaixo

In [77]:
%%sql

SELECT cl.*, po.*
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
LIMIT 5

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
6,Ana Júlia Vieira,aragaokamilly@bol.com.br,18,62,prod 62,9.92
14,Davi Lucas Aragão,ecunha@uol.com.br,44,16,prod 16,2.86
3,Alexandre Santos,fpereira@yahoo.com.br,29,11,prod 11,7.78
14,Davi Lucas Aragão,ecunha@uol.com.br,44,12,prod 12,6.65
9,Beatriz Pinto,ana-sophia02@ig.com.br,54,8,prod 08,4.23


A tabela de compras traz consigo o ID do cliente que fez a compra e o ID do produto
comprado, ocupando assim muito menos espaço do que se eu tivesse uma tabela com
todos os dados.

> NOTA: Quando nas colunas selecionadas faço algo do tipo alias.* eu estou trazendo todas as colunas daquela tabel

> NOTA: Em 99% dos casos, independetemente do intuito do SELECT, o relacionamento entre as tabelas vai utilizar as mesmas colunas que usou em SELECTs anteriores

## Exercício 4

Traga 10 compras do produto `prod 02`

In [78]:
%%sql

SELECT cl.*, po.*
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade,id_1,produto,preço
18,Theo Campos,azevedorodrigo@uol.com.br,43,2,prod 02,7.56
7,Igor Dias,caua15@uol.com.br,28,2,prod 02,7.56


# Distinct

Muitas vezes as nossas consultas nos trazem resultados repetidos, quando a repetição
não nos importa, por exemplo, mesmo na nossa base tendo 20 clientes, não necessariamente
todos efetuaram compras, se eu quiser saber todos que o fizeram posso fazer:

In [79]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID

 * sqlite:///TesteSQL.db
Done.


nome
Ana Júlia Vieira
Davi Lucas Aragão
Alexandre Santos
Beatriz Pinto
Rafaela Rezende
Manuela Lima
Sr. Nathan Novaes
Vitor Gabriel Cavalcanti
Breno Ribeiro
Luiz Otávio Campos


Como podemos ver, apesar de eu ter feito {{N_COMPRAS}} compras, o meu resultado
não traz essa quantidade de linhas, traz exatamente a quantidade de clientes 
que fizeram compras.

## Exercício 5

Liste todos clientes que compraram o produto `prod 02`

In [80]:
%%sql

SELECT DISTINCT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE produto = 'prod 02'


 * sqlite:///TesteSQL.db
Done.


nome
Igor Dias
Theo Campos


# Redução

No SQL também são comuns funções de redução, elas percorrem todas as linhas de uma
determinada coluna aplicando uma função de redução (somatório, contagem e etc.)
e retornam somente uma linha ao invés de todas as linhas lidas por ela, por exemplo
se eu quiser saber quantas compras do `prod 02` foram feitas, posso fazer:

In [81]:
%%sql

SELECT count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
WHERE po.produto = 'prod 02'

 * sqlite:///TesteSQL.db
Done.


count(*)
2


Existem outras funções como SUM, AVG e etc., você pode sempre consultar a 
[documentação oficial](https://www.sqlite.org/lang_aggfunc.html) para ver 
todas elas

## Exercício 6

Qual foi a receita total?

In [82]:
%%sql

SELECT round(sum(po.preço)) AS receita_total
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


receita_total
2602.0


# Group by

As funções de agregação ficam ainda mais fortes quando agrupamos valores.
Isto é, elas mostram todo o seu potencial quando criamos pequenos grupos de dados,
ao invés de contarmos por exemplo todas as vendas do dia, podemos aplicar contagem
produto a produto com somente uma query, como mostrado abaixo:

In [83]:
%%sql

SELECT po.produto, count(*)
FROM compras AS co
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,count(*)
prod 00,4
prod 01,3
prod 02,2
prod 03,7
prod 04,2
prod 05,8
prod 06,5
prod 07,3
prod 08,3
prod 09,8


Dessa forma, consigo fazer uma contagem diferente para cada produto, sem precisar
fazer diversas querys, algo muito útil.

## Exercício 7

Quanto cada cliente gastou?

In [84]:
%%sql

SELECT cl.nome, round(sum(po.preço)) as gasto
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,gasto
Alexandre Santos,116.0
Ana Júlia Vieira,82.0
Beatriz Pinto,142.0
Breno Ribeiro,137.0
Davi Lucas Aragão,120.0
Dr. Thomas Almeida,144.0
Dra. Bruna das Neves,125.0
Igor Dias,155.0
João Vitor Fernandes,135.0
Kamilly Silva,169.0


# Order by

Muitas vezes queremos os dados em ordem específicas, por exemplo, ao visitar uma
loja online, você não espera ver produtos caros e desinteressantes sendo exibidos
para você aleatoriamente, você espera ver em uma ordem lógica, sejam os mais
baratos primeiros, os mais relevantes para o seu perfil, ou até alfabéticamente.


Tudo isso pode ser feito utilizando o comando `ORDER BY`, mas misturado com
outras operações como o `LIMIT` ele pode fazer coisas ainda melhores, como
por exemplo, trazer os 10 produtos mais caros:

In [85]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 95,9.94
prod 62,9.92
prod 26,9.89
prod 34,9.73
prod 77,9.72
prod 49,9.68
prod 96,9.49
prod 25,9.44
prod 92,9.39
prod 48,9.33


Com isso podemos ver o poder do `ORDER BY` + `LIMIT` para trazer os N primeiros
registros de uma determinada métrica

## Exercício 8

Traga os 10 clientes que mais gastaram

In [86]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10

 * sqlite:///TesteSQL.db
Done.


nome
Sra. Maria Alice Araújo
Ana Júlia Vieira
Lavínia Vieira
Theo Campos
Alexandre Santos
Manuela Lima
Davi Lucas Aragão
Vitor Gabriel Cavalcanti
Srta. Mariane da Mata
Dra. Bruna das Neves


O order by junto do limit é extremamente importante para fazer paginação de dados,
por exemplo, novamente ao acessar o site de compras, os produtos não são todos
exibidos de uma vez para você, do contrário, a memória do seu computador iria
para o espaço, para evitar isso, tem aqueles botões na parte de baixo para ver
a próxima página, e com isso, você pode avançar e recuar.

Digamos que um cliente viu os 10 produtos mais caros e não se interessou por
nenhum e quer ver a próxima página, ele pode fazer:

In [87]:
%%sql

SELECT produto, preço
FROM produtos
ORDER BY preço DESC
LIMIT 10,10

 * sqlite:///TesteSQL.db
Done.


produto,preço
prod 65,9.19
prod 15,9.0
prod 27,8.97
prod 30,8.93
prod 60,8.93
prod 19,8.73
prod 68,8.72
prod 83,8.59
prod 28,8.53
prod 55,8.35


O primeiro 10, representa de onde começa e o segundo quantos registros vai trazer.
Você pode mexer nos valores acima e testar a vontade para ver como muda.

## Exercício 9

Traga o 11º cliente que mais gastou

In [88]:
%%sql

SELECT cl.nome
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome
order by sum(po.preço)
LIMIT 10,01

 * sqlite:///TesteSQL.db
Done.


nome
Nathan da Mota


# Funções janeladas

As funções janeladas ou funções analíticas como também são conhecidas, são funções
que se aplicam a todas a coluna trazendo métricas como qual posição essa pessoa está
entre as que mais gastaram, ou quantas pessoas no mínimo foram necessárias para 
alcançar até 25% da receita total.

In [89]:
%%sql

SELECT produto, preço, rank() over (order by preço desc) as posição
FROM produtos
order by posição
limit 10

 * sqlite:///TesteSQL.db
Done.


produto,preço,posição
prod 95,9.94,1
prod 62,9.92,2
prod 26,9.89,3
prod 34,9.73,4
prod 77,9.72,5
prod 49,9.68,6
prod 96,9.49,7
prod 25,9.44,8
prod 92,9.39,9
prod 48,9.33,10


Você pode usar as funções janeladas quando lhe for conveniente, mas não serão
obrigatórias em nenhum exercício

## Exercício 10

Qual foi o produto que menos vendeu?

In [90]:
%%sql
SELECT produto, rank() over (order by sum(po.preço) desc ) as menos_vendido
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


produto,menos_vendido
prod 62,1


# Operadores lógicos comuns

Alguns operadores lógicos bem famosos são o `=`, `>`, `<`, `>=`, `<=`, `!=`,
mas existem outros menos comuns, como o `LIKE` e `IN`.

O primeiro, serve para fazer pesquisas simples dentro de texto, ele irá verificar
caracter a caracter se a string informada bate com o padrão, de forma mais prática

```SQL
'abc' LIKE 'abc' -- Verdadeiro

'abcd' LIKE 'abc' -- Falso

'cbc' LIKE 'abc' -- Falso

'dabc' LIKE 'abc' -- Falso
```

Até aí é exatamente igual ao operador `=`, mas quando adicionamos os wildcards
`%` e `_` que o `LIKE` mostra o seu valor.

O wildcard `_` pode ser usado para representar qualquer caracter, por exemplo:

```SQL
'abc' LIKE 'ab_' -- Verdadeiro
'abd' LIKE 'ab_' -- Verdadeiro
'ab0' LIKE 'ab_' -- Verdadeiro
'ab ' LIKE 'ab_' -- Verdadeiro
'abcd' LIKE 'ab_' -- Falso
'cbc' LIKE 'ab_' -- Falso
'ab' LIKE 'ab_' -- Falso
'dabc' LIKE 'ab_' -- Falso
```

Já o wildcard `%` pode ser usado para representar qualquer sequência de caracteres,
por exemplo:

```SQL
'abc' LIKE 'ab%' -- Verdadeiro
'abd' LIKE 'ab%' -- Verdadeiro
'ab0' LIKE 'ab%' -- Verdadeiro
'ab ' LIKE 'ab%' -- Verdadeiro
'abcd' LIKE 'ab%' -- Verdadeiro
'cbc' LIKE 'ab%' -- Falso
'ab' LIKE 'ab%' -- Verdadeiro
'dabc' LIKE 'ab%' -- Falso
```

Usando o nosso exemplo, vamos pegar todos os clientes que tenham e-mail uol:

In [91]:
%%sql

SELECT *
FROM CLIENTES
WHERE EMAIL LIKE '%@uol.com.br'

 * sqlite:///TesteSQL.db
Done.


id,nome,email,idade
2,Luiz Otávio Campos,bcunha@uol.com.br,25
5,Manuela Lima,hferreira@uol.com.br,57
7,Igor Dias,caua15@uol.com.br,28
12,Rafaela Rezende,nogueiraemanuel@uol.com.br,28
14,Davi Lucas Aragão,ecunha@uol.com.br,44
16,Sra. Maria Alice Araújo,franciscocardoso@uol.com.br,53
18,Theo Campos,azevedorodrigo@uol.com.br,43


Já o operador `IN` serve para verificar se o valor está dentro de um cojunto, por exemplo

```SQL
1 in (1, 2, 3) -- Verdadeiro
2 in (1, 2, 3) -- Verdadeiro
3 in (1, 2, 3) -- Verdadeiro
4 in (1, 2, 3) -- Falso
(1, 2) in ((1, 2), (3, 4)) -- Verdadeiro
(1, 3) in ((1, 2), (3, 4)) -- Falso
```

In [92]:
%%sql

SELECT *
FROM PRODUTOS
WHERE PRODUTO IN ('prod 01', 'prod 20', 'batata')

 * sqlite:///TesteSQL.db
Done.


id,produto,preço
1,prod 01,7.85
20,prod 20,0.32


# Subquerys

Subquerys são utilizadas para fazer extrações mais complexas, quando para 
alcançar um resultado, é necessário outro resultado, digamos, listar todos os
produtos comprados pelo cliente que mais gastou, primeiro precisamos encotrar o 
cliente que mais gastou e depois os produtos que ele comprou.

Podemos separar isso em duas querys:

1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou

E elas ficariam como:

In [93]:
%%sql

-- Selecionando o cliente que mais gastou

SELECT cl2.ID, cl2.nome, sum(po2.preço) AS GASTO
FROM clientes cl2
JOIN compras AS co2
ON co2.ID_CLIENTE = cl2.ID
JOIN produtos as po2
ON co2.ID_PRODUTO = po2.ID
group by cl2.nome, cl2.id
ORDER BY gasto DESC
LIMIT 1

 * sqlite:///TesteSQL.db
Done.


id,nome,GASTO
2,Luiz Otávio Campos,188.81


In [94]:
%%sql

-- Selecionar todos os produtos que umm cliente comprou

SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (5)

 * sqlite:///TesteSQL.db
Done.


produto
prod 13
prod 64
prod 72
prod 17
prod 70
prod 22
prod 39
prod 93
prod 11
prod 37


In [95]:
%%sql

-- Juntando as duas:
    
SELECT DISTINCT po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
WHERE co.ID_CLIENTE IN (
    SELECT cl2.ID
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
)

 * sqlite:///TesteSQL.db
Done.


produto
prod 48
prod 77
prod 23
prod 74
prod 36
prod 43
prod 05
prod 19
prod 60
prod 46


Se eu quisesse exibir o nome do cliente, eu poderia fazer mais um join pegando
o nome dele na tabela cliente, mas nesse caso, eu estou pegando um dado que eu
consigo facilmente na subquery, existem formas de trazer dados lá de dentro,
como por  exemplo:

In [96]:
%%sql

    
SELECT tb.nome, po.produto
FROM produtos as po
JOIN compras AS co
ON co.ID_PRODUTO = po.ID
join (
    SELECT cl2.ID, cl2.nome
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by cl2.nome, cl2.id
    ORDER BY sum(po2.preço) DESC
    LIMIT 1
) tb on co.ID_CLIENTE = tb.ID
group by tb.nome, po.produto

 * sqlite:///TesteSQL.db
Done.


nome,produto
Luiz Otávio Campos,prod 03
Luiz Otávio Campos,prod 05
Luiz Otávio Campos,prod 06
Luiz Otávio Campos,prod 07
Luiz Otávio Campos,prod 08
Luiz Otávio Campos,prod 13
Luiz Otávio Campos,prod 17
Luiz Otávio Campos,prod 19
Luiz Otávio Campos,prod 21
Luiz Otávio Campos,prod 23


Não existe limite de quantidade de subquerys que você pode fazer, sejam uma duas
ou várias, fazendo uma verdadeira boneca russa, mas use com atenção, verifique
sempre se realmente é necessário.

Se por exemplo, eu quisesse saber quantos clientes compraram os mesmos
produtos que a pessoa que mais gastou comprou


1. Selecionar o cliente que mais gastou
2. Selecionar todos os produtos que um cliente comprou
3. Contar quantos clientes compraram um produto

Como a query 1 e 2 já são antigas, vou fazer a 3

In [97]:
%%sql

-- Contar quantos clientes compraram um produto

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (5)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 05,6


In [98]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos po on po.ID = co.ID_PRODUTO
WHERE co.ID_PRODUTO IN (
    SELECT DISTINCT po2.id
    FROM produtos as po2
    JOIN compras AS co2
    ON co2.ID_PRODUTO = po2.ID
    WHERE co2.ID_CLIENTE IN (
        SELECT cl3.ID
        FROM clientes cl3
        JOIN compras AS co3
        ON co3.ID_CLIENTE = cl3.ID
        JOIN produtos as po3
        ON co3.ID_PRODUTO = po3.ID
        group by cl3.nome, cl3.id
        ORDER BY sum(po3.preço) DESC
        LIMIT 1
    )
)
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,5
prod 05,6
prod 06,5
prod 07,3
prod 08,3
prod 13,5
prod 17,6
prod 19,4
prod 21,4
prod 23,6


Mas novamente podemos simplificar usando join ao invés de where, lembre-se,
quanto menos vezes você repetir o nome de uma tabela, melhor

In [99]:
%%sql

SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN (
    SELECT cl3.ID
    FROM clientes cl3
    JOIN compras AS co3
    ON co3.ID_CLIENTE = cl3.ID
    JOIN produtos as po3
    ON co3.ID_PRODUTO = po3.ID
    group by cl3.nome, cl3.id
    ORDER BY sum(po3.preço) DESC
    LIMIT 1
) q1
JOIN(
    SELECT co2.id_PRODUTO, co2.ID_CLIENTE
    FROM compras AS co2
    GROUP BY co2.id_PRODUTO, co2.ID_CLIENTE
) q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,5
prod 05,6
prod 06,5
prod 07,3
prod 08,3
prod 13,5
prod 17,6
prod 19,4
prod 21,4
prod 23,6


# With

Muitas vezes fica confuso ficar colocando várias subquerys no meio da query 
pricipal, ou você quer repetir ela, não quer ter que trocar alias e etc.

Para resolver esses problemas, ou até outros, consulte a 
[documentação](https://www.sqlite.org/lang_with.html) para ver
tudo o que ele é capaz, existe o `WITH`.

In [100]:
%%sql

WITH q1 AS (
    SELECT cl.ID
    FROM clientes cl
    JOIN compras AS co
    ON co.ID_CLIENTE = cl.ID
    JOIN produtos as po
    ON co.ID_PRODUTO = po.ID
    group by cl.nome, cl.id
    ORDER BY sum(po.preço) DESC
    LIMIT 1
),
q2 AS (
    SELECT co.id_PRODUTO, co.ID_CLIENTE
    FROM compras AS co
    GROUP BY co.id_PRODUTO, co.ID_CLIENTE
)
SELECT po.produto, count(DISTINCT co.ID_CLIENTE)
FROM compras co
JOIN produtos as po on co.ID_PRODUTO = po.ID
JOIN q1
JOIN q2 ON co.ID_PRODUTO = q2.ID_PRODUTO AND q2.ID_CLIENTE = q1.ID
GROUP by po.produto

 * sqlite:///TesteSQL.db
Done.


produto,count(DISTINCT co.ID_CLIENTE)
prod 03,5
prod 05,6
prod 06,5
prod 07,3
prod 08,3
prod 13,5
prod 17,6
prod 19,4
prod 21,4
prod 23,6


## Exercício 11

Traga todos os clientes que compraram o produto que menos vendeu

In [101]:
%%sql


    
SELECT DISTINCT cl.nome, po.produto      #todos os clientes que compraram
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID 
WHERE co.ID_PRODUTO IN (             #colocar o mesmo no select
SELECT  co2.ID_PRODUTO              #produto que menos vendeu
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by po2.produto
    order by  count(*)
    limit 1
)

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#todos": syntax error
[SQL: SELECT DISTINCT cl.nome, po.produto      #todos os clientes que compraram
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID 
WHERE co.ID_PRODUTO IN (             #colocar o mesmo no select
SELECT  co2.ID_PRODUTO              #produto que menos vendeu
    FROM clientes cl2
    JOIN compras AS co2
    ON co2.ID_CLIENTE = cl2.ID
    JOIN produtos as po2
    ON co2.ID_PRODUTO = po2.ID
    group by po2.produto
    order by  count(*)
    limit 1
)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Exercício 12

Traga quanto gastou cada cliente que comprou o produto que menos vendeu

In [102]:
%%sql

WITH quantidade_de_produto_mv as (         
	SELECT count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	order by qtd
	limit 1	
),
produto_que_menos_vendeu AS (                                 #produto que menos vendeu
	SELECT co.ID_PRODUTO, count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	having qtd = (SELECT qtd from quantidade_de_produto_mv)
),
oq_cd_1_comprou as (                               #o que cada cliente comprou
	SELECT
		cl.nome, 
		co.id_cliente,
		po.produto,
		co.id_produto,
		po.preço 
	FROM clientes cl
	JOIN compras AS co ON co.ID_CLIENTE = cl.ID
	JOIN produtos as po ON co.ID_PRODUTO = po.ID
), quanto_cada_um_gastou as (                          #quanto gastou cada cliente
   SELECT 
    id_cliente,
    nome,
 	sum(preço) as gasto
   FROM oq_cd_1_comprou
   group by id_cliente, nome
 )
 , quem_comprou_menos_vendeu as (               #quem comprou o que menos vendeu
	select 
 	DISTINCT nome, id_cliente 
 	FROM oq_cd_1_comprou as occ
 	JOIN produto_que_menos_vendeu as pmv ON occ.id_produto = pmv.id_produto
 )
  SELECT                                  #quanto gastou cada cliente que comprou o produto que menos vendeu
  qcmv.nome, qcmv.id_cliente, gasto
  FROM quanto_cada_um_gastou as qcug
  JOIN quem_comprou_menos_vendeu as qcmv ON   qcug.id_cliente = qcmv.id_cliente

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#produto": syntax error
[SQL: WITH quantidade_de_produto_mv as (         
	SELECT count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	order by qtd
	limit 1	
),
produto_que_menos_vendeu AS (                                 #produto que menos vendeu
	SELECT co.ID_PRODUTO, count(*) as qtd
	FROM compras AS co
	group by co.id_produto
	having qtd = (SELECT qtd from quantidade_de_produto_mv)
),
oq_cd_1_comprou as (                               #o que cada cliente comprou
	SELECT
		cl.nome, 
		co.id_cliente,
		po.produto,
		co.id_produto,
		po.preço 
	FROM clientes cl
	JOIN compras AS co ON co.ID_CLIENTE = cl.ID
	JOIN produtos as po ON co.ID_PRODUTO = po.ID
), quanto_cada_um_gastou as (                          #quanto gastou cada cliente
   SELECT 
    id_cliente,
    nome,
 	sum(preço) as gasto
   FROM oq_cd_1_comprou
   group by id_cliente, nome
 )
 , quem_comprou_menos_vendeu as (               #quem comprou o que menos ve

## Exercício 13

Quanto em média os clientes gastaram?

In [103]:
%%sql

SELECT sum(po.preço)/count(distinct co.id_cliente) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID

 * sqlite:///TesteSQL.db
Done.


média
130.10100000000008


## Exercício 14

Quanto em média gastou cada cliente?

In [104]:
%%sql

SELECT cl.nome, sum(po.preço)/count(*) as média
FROM clientes cl 
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by cl.nome


 * sqlite:///TesteSQL.db
Done.


nome,média
Alexandre Santos,5.026521739130435
Ana Júlia Vieira,3.9180952380952374
Beatriz Pinto,5.454615384615385
Breno Ribeiro,5.286153846153846
Davi Lucas Aragão,5.444545454545454
Dr. Thomas Almeida,6.004583333333334
Dra. Bruna das Neves,5.961428571428571
Igor Dias,5.354827586206897
João Vitor Fernandes,5.191538461538462
Kamilly Silva,6.242222222222221


## Exercício 15

Quantos clientes tem cada faixa etária (Agrupe por faixa de 10 anos)

i.e.: \[18 -> 19\], \[20 -> 29\], \[30 -> 39\], ...

In [105]:
%%sql

SELECT cast(idade/10 AS integer)*10 as faixa_etaria, count(*)
FROM clientes cl
group by faixa_etaria

 * sqlite:///TesteSQL.db
Done.


faixa_etaria,count(*)
10,1
20,6
30,2
40,6
50,5


## Exercício 16

Qual é o gasto médio para cada faixa etária?

In [50]:
%%sql

SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria

 * sqlite:///TesteSQL.db
Done.


faixa_etaria,gasto_médio
10,136.47000000000008
20,106.37333333333329
30,137.11
40,123.97666666666667
50,138.46999999999994
60,146.39000000000001


## Exercício 17

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto médio?

In [51]:
%%sql

SELECT po.produto, count(*)            #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5


SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio   # MAIOR GASTO MÉDIO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_médio DESC
limit 1




 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#5": syntax error
[SQL: SELECT po.produto, count(*) #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5


SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço)/count(distinct co.id_cliente) as gasto_médio   # MAIOR GASTO MÉDIO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_médio DESC
limit 1]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


## Exercício 18

Quais foram os 5 produtos que mais venderam na faixa etária que teve maior
gasto absoluto?

In [106]:
%%sql

SELECT po.produto, count(*)            #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5




SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço) as gasto_absoluto   #MAIOR GASTO ABSOLUTO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_absoluto DESC
limit 1

 * sqlite:///TesteSQL.db
(sqlite3.OperationalError) near "#5": syntax error
[SQL: SELECT po.produto, count(*) #5 produtos que mais venderam
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by po.produto
order by  count(*) DESC
limit 5




SELECT cast(idade/10 AS integer)*10 as faixa_etaria, sum(po.preço) as gasto_absoluto   #MAIOR GASTO ABSOLUTO
FROM clientes cl
JOIN compras AS co
ON co.ID_CLIENTE = cl.ID
JOIN produtos as po
ON co.ID_PRODUTO = po.ID
group by faixa_etaria
order by gasto_absoluto DESC
limit 1]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
